# Imports

In [1]:
import dijkstra
import astar
import pandas
import data_loader
from timer import Timer
import cli_utils

# Loading data

In [2]:
graph= data_loader.load_data_to_graph(normalize_coordinates=True)
graph_dict = graph.graph_dict

# Test data

In [3]:
bus_stops = [
    ('wrocławski park przemysłowy','pl. grunwaldzki'),
    ('grabiszyńska','pl. daniłowskiego'),
    ('kwiska','pl. grunwaldzki'),
    ('krzyki','leśnica'),
    ('leśnica','biskupin'),
    ('wrocławski park przemysłowy','dworzec autobusowy'),
    ('fat','osiedle sobieskiego')
]
bus_stops_test_data = [(graph_dict[start_stop], graph_dict[end_stop]) for start_stop, end_stop in bus_stops]

start_times = ['9:00','9:15','9:30', '2:00', '2:15', '2:30']
start_times_test_data = [cli_utils.normalize_input_time(time) for time in start_times]

# Data analysis

In [6]:
data = []
configs = {
    'Dijkstra for time' : lambda start_stop, end_stop, start_time: dijkstra.shortest_path(graph_dict=graph_dict, start_stop=start_stop.name.lower(), goal_stop=end_stop.name.lower(), start_time=start_time, cost_fn=dijkstra.DEFAULT_COST_FUNCTION),
    'A* for time' : lambda start_stop, end_stop, start_time: astar.astar(start=start_stop, goal=end_stop, start_time=start_time, cost_fn=astar.TIME_COST_FUNCTION, heuristic_fn=astar.TIME_HEURISTIC),
    'A* for line changes' : lambda start_stop, end_stop, start_time: astar.astar(start=start_stop, goal=end_stop, start_time=start_time, cost_fn=astar.LINE_CHANGE_COST_FUNCTION, heuristic_fn=astar.LINE_CHANGE_HEURISTIC)
}

timer = Timer()

for start_time in start_times_test_data:
    for start_stop, end_stop in bus_stops_test_data:
        for config_name, function in configs.items():
            
            cost, _ = timer.run(lambda : function(start_stop, end_stop, start_time))
            computation_time = timer.elapsed_time
            
            record = config_name, start_stop, end_stop, cli_utils.convert_normalized_time(start_time), cost, computation_time
            data.append(record)

columns = ['Algorithm', 'Start stop', 'End stop', 'Start time', 'Cost', 'Computation time']

statistics = pandas.DataFrame(data, columns=columns)
statistics

In [10]:
time_optimization_statistics = statistics[statistics['Algorithm'] != 'A* for line changes']


pivoted_time_optimization_statistics = time_optimization_statistics.pivot_table(index=['Start stop', 'End stop', 'Start time'], columns='Algorithm', values=['Cost', 'Computation time'])
pivoted_time_optimization_statistics

Computation time  \
Algorithm                                                       A* for time   
Start stop                  End stop            Start time                    
FAT                         OSIEDLE SOBIESKIEGO 02:00              0.619618   
                                                02:15              0.448001   
                                                02:30              0.342819   
                                                09:00              0.294347   
                                                09:15              0.298398   
                                                09:30              0.272847   
Grabiszyńska                pl. Daniłowskiego   02:00              0.569070   
                                                02:15              0.384008   
                                                02:30              0.374168   
                                                09:00              0.262994   
                                                09:15              0.145774   
                                                09:30              0.127097   
KRZYKI                      LEŚNICA             02:00              0.665482   
                                                02:15              0.524109   
                                                02:30              0.593156   
                                                09:00              0.206374   
                                                09:15              0.266526   
                                                09:30              0.242228   
Kwiska                      PL. GRUNWALDZKI     02:00              0.320946   
                                                02:15              0.331256   
                                                02:30              0.307798   
                                                09:00              0.181845   
                                                09:15              0.155881   
                                                09:30              0.161723   
LEŚNICA                     BISKUPIN            02:00              0.558099   
                                                02:15              0.474912   
                                                02:30              0.406632   
                                                09:00              0.236413   
                                                09:15              0.252322   
                                                09:30              0.270149   
Wrocławski Park Przemysłowy DWORZEC AUTOBUSOWY  02:00              0.064854   
                                                02:15              0.068858   
                                                02:30              0.064000   
                                                09:00              0.127742   
                                                09:15              0.079074   
                                                09:30              0.109292   
                            PL. GRUNWALDZKI     02:00              0.273251   
                                                02:15              0.263030   
                                                02:30              0.227439   
                                                09:00              0.134323   
                                                09:15              0.089826   
                                                09:30              0.089085   

                                                                              \
Algorithm                                                  Dijkstra for time   
Start stop                  End stop            Start time                     
FAT                         OSIEDLE SOBIESKIEGO 02:00               1.113476   
                                                02:15               0.988176   
                                                02:30               0.880000   
                       